In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir("C:/Personal/Coding/Quant Project/Research")
df = pd.read_csv('Data/ICICIBANK_Feature.csv', index_col=0, parse_dates=True)
df.index.name = "timestamp"
from src.strategy import backtest_mean_reversion_existing_features


In [ ]:
df=backtest_mean_reversion_existing_features(df)

In [ ]:
df

In [ ]:
trade_count = (df['position'].diff().abs() > 0).sum()
print(trade_count)
trades_per_day = trade_count / (len(df) / 390)
print(trades_per_day)

In [ ]:
sharpe = (
    df['strategy_return'].mean() /
    df['strategy_return'].std()
) * np.sqrt(252 * 390)

print(f"Sharpe: {sharpe:.2f}")


In [ ]:
net_profit_log = df['cum_strategy_net'].iloc[-1]
print(net_profit_log)

In [ ]:
net_profit_pct = np.exp(df['cum_strategy_net'].iloc[-1]) - 1
print(f"Net Profit After Costs: {net_profit_pct:.2%}")

In [ ]:
gross_pct = np.exp(df['cum_strategy'].iloc[-1]) - 1
net_pct   = np.exp(df['cum_strategy_net'].iloc[-1]) - 1

print(f"Gross Return: {gross_pct:.2%}")
print(f"Net Return:   {net_pct:.2%}")
print(f"Cost Impact:  {(gross_pct - net_pct):.2%}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir("C:/Personal/Coding/Quant Project/Research")
df = pd.read_csv('Data/ICICIBANK_Feature.csv', index_col=0, parse_dates=True)
df.index.name = "timestamp"
from src.strategy import backtest_mean_reversion_existing_features

In [ ]:
df=backtest_mean_reversion_existing_features(df)


In [ ]:
df

In [ ]:
trade_count = (df['position'].diff().abs() > 0).sum()
print(trade_count)
trades_per_day = trade_count / (len(df) / 390)
print(trades_per_day)

In [ ]:
sharpe = (
    df['strategy_return'].mean() /
    df['strategy_return'].std()
) * np.sqrt(252 * 390)

print(f"Sharpe: {sharpe:.2f}")

In [ ]:
df_result=df

In [ ]:

print("="*60)
print("DIAGNOSTIC REPORT")
print("="*60)

# 1. How much are filters cutting trades?
print("\n1. FILTER IMPACT")
print(f"Bars passing filters: {df_result['trade_filter'].mean():.1%}")
total_trades = df_result['trade'].sum() // 2  # Divide by 2 (entry+exit)
print(f"Total round-trip trades: {total_trades}")
print(f"Trades per day: {total_trades / len(df_result.index.date.unique()):.1f}")

# 2. What's killing performance?
print("\n2. PERFORMANCE BREAKDOWN")
gross = df_result['strategy_return'].sum()
costs = df_result['transaction_cost'].sum()
net = df_result['strategy_return_net'].sum()

print(f"Gross return: {gross:.4f}")
print(f"Total costs:  {costs:.4f}")
print(f"Net return:   {net:.4f}")
print(f"Cost ratio:   {costs/abs(gross) if gross != 0 else 'N/A':.1%}")

# 3. Per-trade economics
if total_trades > 0:
    print(f"\nAvg gross per trade: {gross/total_trades:.6f}")
    print(f"Avg cost per trade:  {0.0005:.6f}")
    print(f"Avg net per trade:   {net/total_trades:.6f}")

# 4. Win rate
trade_pnl = df_result.groupby('trade_block')['strategy_return_net'].sum()
trade_pnl = trade_pnl[trade_pnl != 0]  # Remove flat periods
print(f"\n4. WIN RATE")
print(f"Winning trades: {(trade_pnl > 0).sum()} / {len(trade_pnl)}")
print(f"Win rate: {(trade_pnl > 0).mean():.1%}")
print(f"Avg winner: {trade_pnl[trade_pnl > 0].mean():.6f}")
print(f"Avg loser:  {trade_pnl[trade_pnl < 0].mean():.6f}")

# 5. Holding time stats
hold_times = df_result.groupby('trade_block')['holding_time'].max()
hold_times = hold_times[hold_times > 0]
print(f"\n5. HOLDING TIME")
print(f"Avg hold: {hold_times.mean():.1f} bars")
print(f"Max hold hits (30 bars): {(hold_times >= 30).sum()}")